### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [3]:
# read the data into pandas dataframes
MN_T00_GD_226a = pd.read_csv("lca/cleaned/226_MN_T00_GD_lca.csv")

# delete any Metazoa hits because they're trypsin
MN_T00_GD_226b = MN_T00_GD_226a[MN_T00_GD_226a.kingdom != 'Metazoa']

# keep only entries to the phylum level
MN_T00_GD_226 = MN_T00_GD_226b[MN_T00_GD_226b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(MN_T00_GD_226))

MN_T00_GD_226.head(6)

# of phylum peptides =  22


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
4,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,QEFLNAAK,Phorcysia thermohydrogeniphila,Bacteria,NaN,NaN,NaN,Aquificae,NaN,NaN,Aquificae,...,Phorcysia,NaN,NaN,NaN,Phorcysia thermohydrogeniphila,NaN,NaN,NaN,NaN,NaN
20,VVEVSLPR,Planctomycetales,Bacteria,NaN,NaN,NaN,Planctomycetes,NaN,NaN,Planctomycetia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,ETCEPLK,[Candida] arabinofermentans NRRL YB-2248,Eukaryota,Fungi,Dikarya,NaN,Ascomycota,Saccharomycotina,NaN,Saccharomycetes,...,Ogataea,NaN,NaN,NaN,[Candida] arabinofermentans,NaN,[Candida] arabinofermentans NRRL YB-2248,NaN,NaN,NaN
60,MHLMHLTK,Dorea,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,Dorea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [4]:
MN_T00_GD_226_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/226_MN_T00_GD_DN50_ILnaafs.csv")
MN_T00_GD_226_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/226B_MN_T00_GD_DN50_ILnaafs.csv")
#MN_T00_GD_226_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/226C_MN_T00_GD_DN50_ILnaafs.csv")


frames = [MN_T00_GD_226_NAAFa, MN_T00_GD_226_NAAFb]

MN_T00_GD_226_NAAF = pd.concat(frames, sort=False)

MN_T00_GD_226_NAAF.set_index('stripped_peptide')
MN_T00_GD_226_NAAF = MN_T00_GD_226_NAAF.loc[:, ~MN_T00_GD_226_NAAF.columns.str.contains('^Unnamed')]

MN_T00_GD_226_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(MN_T00_GD_226_NAAF))

print('column names:', MN_T00_GD_226_NAAF.columns)

MN_T00_GD_226_NAAF.head()

# of total peptides =  265
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,VLEGNEQFLNAAK,1970000.0,1.515385e+05
1,LSSPATLNSR,53600000.0,5.360000e+06
2,LSSPATLNSR,53600000.0,5.360000e+06
3,LSSPATLNSR,53600000.0,5.360000e+06
4,LSSPATLNSR,53600000.0,5.360000e+06


In [5]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(MN_T00_GD_226["peptide"], MN_T00_GD_226_NAAF["peptide"])

print(over)

['ETCEPLK' 'LATVLSPR' 'NNFASWR' 'QEFLNAAK' 'TTTVSLPR' 'VATVSPLR'
 'VVEVSLPR']


In [6]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

MN_T00_GD_226.set_index('peptide', inplace=True)
MN_T00_GD_226_NAAF.set_index('peptide', inplace=True)

MN_T00_GD_226_Phy = MN_T00_GD_226.join(MN_T00_GD_226_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(MN_T00_GD_226_Phy))

MN_T00_GD_226_Phy.head()

# of total phylum-level peptides =  37


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
EHCYVLDFSPNR,Actinomycetia,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ETCEPLK,[Candida] arabinofermentans NRRL YB-2248,Eukaryota,Fungi,Dikarya,NaN,Ascomycota,Saccharomycotina,NaN,Saccharomycetes,NaN,...,NaN,NaN,[Candida] arabinofermentans,NaN,[Candida] arabinofermentans NRRL YB-2248,NaN,NaN,NaN,116000.0,1.657143e+04
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42900000.0,5.362500e+06
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42900000.0,5.362500e+06
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42900000.0,5.362500e+06


In [7]:
# write to a csv

MN_T00_GD_226_Phy.to_csv("lca/NAAF/MN_T00_GD_226_Phy_naaf.csv")